## Exploratory data analysis (EDA) of behvaioral data

Load necessary libraries

In [4]:
import pandas as pd
import numpy as np
import sys
sys.path.append("src")

from src.load_files import load_posts_ratings

Load data

In [ ]:
# Load metadata (csv)
metadata = pd.read_csv('path/to/metadata.csv')

# Load political attitudes data (csv)
attitudes_data = pd.read_csv('path/to/political_attitudes.csv')

# Load political polarization data (csv)
polarization_data = pd.read_csv('path/to/polarization_data.csv')

# Load posts ratings data (csv)
posts_ratings = load_posts_ratings('path/to/posts_ratings_directory')


